In [1]:
import os
import json
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())
pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
df = pl.read_parquet(os.path.join("data","cnb_ceska_proza.parquet")).rename({'autorstvo_kod':'100_7'})
df = pl.concat([df, pl.read_parquet(os.path.join("data","cnb_ceska_poezie.parquet"))])

In [5]:
spisovatelstvo = df.select(pl.col('100_7')).drop_nulls().to_series().to_list()
spisovatelstvo

['jk01103049',
 'jk01033422',
 'jk01082969',
 'jk01061522',
 'xx0011353',
 'jk01062116',
 'jk01131282',
 'jk01032289',
 'jk01131282',
 'jk01131282',
 'jk01131282',
 'jk01130618',
 'jk01043384',
 'jk01122167',
 'jk01051336',
 'jk01110509',
 'xx0011353',
 'jk01040620',
 'jk01010030',
 'jk01121675',
 'jk01131282',
 'jk01131282',
 'jk01072132',
 'jk01092270',
 'jk01070301',
 'jk01051353',
 'jk01102134',
 'jk01102134',
 'jk01033098',
 'jk01102134',
 'jk01060448',
 'jk01033420',
 'jn20020403005',
 'jk01060448',
 'jk01102378',
 'jk01110509',
 'jk01150903',
 'jk01060448',
 'jk01130907',
 'jk01072743',
 'jk01132742',
 'jk01070301',
 'jk01050232',
 'jk01023318',
 'jk01122167',
 'jk01080116',
 'jk01141531',
 'jk01050232',
 'jx20060531005',
 'jk01050232',
 'jk01051336',
 'jk01050232',
 'jk01050232',
 'jk01062786',
 'jx20060522007',
 'jk01102378',
 'jk01152586',
 'jx20080102031',
 'jk01070301',
 'jk01122167',
 'jk01152586',
 'jk01142266',
 'jk01062870',
 'jx20080102031',
 'jk01131949',
 'jk01072553

In [7]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [9]:
aut = aut.explode("370_c").explode("100_d").explode('100_a').explode('100_7')
aut = aut.filter(pl.col("370_c") == "Česko")

In [10]:
def narozeni(udaje):
    try:
        return int(udaje.split("-")[0])
    except:
        return None

In [13]:
def umrti(udaje):
    try:
        return int(udaje.split("-")[1])
    except:
        return None

In [15]:
chyby = ["Goll, Karl,","Bezděk, Karel,","Pelcová, Ludmila,","Fencl, Jaroslav,"]

In [17]:
aut = aut.filter(pl.col('100_7').is_in(spisovatelstvo))

In [19]:
aut = aut.filter(~pl.col('100_a').is_in(chyby))

In [21]:
aut = aut.with_columns(pl.col("100_d").map_elements(narozeni).alias('narozeni')).with_columns(pl.col("100_d").map_elements(umrti).alias('umrti'))

In [35]:
aut.filter(pl.col('100_a').str.contains('Karel Hynek'))

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,narozeni,umrti
list[str],list[str],list[str],list[str],str,str,str,list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,i64,i64
"[""isni"", ""wikidata""]","[""0000000121031971"", ""Q361265""]","[""1810""]","[""1836""]","""jk01072915""","""Mácha, Karel Hynek,""","""1810-1836""","[""1""]",null,"[""Praha, Česko""]","[""Litoměřice, Česko""]","""Česko""",null,"[""literární tvorba"", ""poezie"", ""próza""]",null,"[""spisovatelé"", ""básníci"", ""prozaici""]","[""muž""]","[""cze""]","[""Maha,"", ""Maha, Karäl Henek,""]","[""1810-1836"", ""1810-1836""]",null,"[""0"", ""1""]",null,null,null,null,null,null,null,null,"[""Narozen 16. 11. 1810 v Praze, zemřel 6. 11. 1836 v Litoměřicích. Spisovatel, básník a prozaik.""]",null,""" cz a22 n 4500""","""jk01072915""",1810,1836
null,null,"[""1977""]",null,"""jo2014807752""","""Brácha, Karel Hynek,""","""1977-""","[""1""]",null,"[""Teplice, Česko""]",null,"""Česko""",null,"[""literatura"", ""školství"", … ""knihovnictví""]",null,"[""básníci"", ""učitelé"", … ""knihovníci""]","[""muž""]","[""cze""]",null,null,null,null,null,null,null,null,"[""Šedivý, Milan,""]","[""Skutečné jméno:""]","[""1""]",null,"[""Narozen 8. 1. 1977 v Teplicích. Básník. Civilním povoláním pedagog, personalista a knihovník.""]",null,""" cz a22 n 4500""","""jo2014807752""",1977,null


In [36]:
aut.drop_nulls(subset=['umrti']).filter(pl.col("narozeni") <= 1836).sort('umrti', descending=True).head(n=10)

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,narozeni,umrti
list[str],list[str],list[str],list[str],str,str,str,list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,i64,i64
"[""isni"", ""wikidata""]","[""000000008194381X"", ""Q362154""]","[""1835""]","[""1923""]","""jk01040997""","""Heyduk, Adolf,""","""1835-1923""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,"[""Hejduk, Adolf,""]","[""1835-1923""]",null,"[""1""]",null,null,null,null,null,null,null,null,"[""Narozen 6. 6. 1835 v Rychmburku u Hlinska, zemřel 6. 2. 1923 v Písku. Básník, člen družiny májové, iniciátor česko-slovenských vztahů.""]","[""http://cs.wikipedia.org/wiki/Adolf_Heyduk""]",""" cz a22 n 4500""","""jk01040997""",1835,1923
null,null,"[""1834""]","[""1921""]","""jk01130591""","""Šmídek, Karel,""","""1834-1921""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 7.9.1834 v Bystřici nad Popelkou, zemřel 4.5.1921 v Brně. Pedagog, práce v oboru pedagogiky, historie a filozofie, překlady z němčiny.""]",null,""" nz a22 n 4500""","""jk01130591""",1834,1921
null,null,"[""1828""]","[""1921""]","""jk01131887""","""Švéda, Jan,""","""1828-1921""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,"[""Šwéda, Jan,""]","[""1828-1921""]",null,"[""1""]",null,null,null,null,null,null,null,null,"[""Narozen 9.2.1828 v Českých Budějovicích, zemřel 7.1.1921 v Prachaticích. Msgre., katolický kněz, náboženská literatura.""]",null,""" cz a22 n 4500""","""jk01131887""",1828,1921
"[""isni"", ""wikidata""]","[""0000000116682462"", ""Q12001298""]","[""1832""]","[""1920""]","""jk01072706""","""Lužická, Věnceslava,""","""1832-1920""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,"[""Lužická, Věnceslava Anna,"", ""Srbová, Anna,""]","[""1832-1920"", ""1832-1920""]","[null, ""Skutečné jméno:""]","[""1"", ""1""]",null,null,null,null,"[""Srbová-Lužická, Anna,""]",null,"[""1""]",null,"[""Narozena 6. 12. 1832 v Hořicích, zemřela 4. 5. 1920 v Praze. Kulturní a organizační pracovnice v ženských spolcích, autorka próz s ženskou tematikou, knih pro mládež a výchovných a popularizačních prací, redaktorka, publicistka.""]",null,""" cz a22 n 4500""","""jk01072706""",1832,1920
null,null,"[""1836""]","[""1920""]","""jk01122364""","""Šafaříková, Paulina,""","""1836-1920""","[""1""]",null,"[""Praha, Česko""]","[""Praha, Česko""]","""Česko""",null,"[""astronomie"", ""popularizace vědy""]",null,null,"[""žena""]","[""cze""]","[""Králová, Paulina,""]","[""1836-1920""]","[""Rodné jméno:""]","[""1""]",null,null,null,null,null,null,null,null,"[""Narozena 11. 4. 1836 v Praze, zemřela 30. 3. 1920 tamtéž. Práce v oboru popularizace astronomie.""]",null,""" cz a22 n 4500""","""jk01122364""",1836,1920
null,null,"[""1832""]","[""1920""]","""jo2009551991""","""Srbová-Lužická, Anna,""","""1832-1920""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Lužická, Věnceslava,""]","[""Pseudonym:""]","[""1""]",null,"[""Narozena 6. 12. 1832 v Hořicích, zemřela 4. 5. 1920 v Praze. Kulturní a organizační pracovnice v ženských spolcích, autorka próz s ženskou tematikou, knih pro mládež a výchovných a popularizačních prací, redaktorka, publicistka.""]",null,""" nz a22 n 4500""","""jo2009551991""",1832,1920
null,null,"[""1836""]","[""1919""]","""jk01021461""","""Černohouz, Jan Nep.,""","""1836-1919""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,"[""Černohouz, Jan Nepomuk,""]","[""1836-1919""]",null,"[""1""]",null,null,null,null,null,null,null,null,"

In [37]:
aut.drop_nulls(subset=['umrti']).filter(pl.col("narozeni") < 1836).sort('umrti', descending=True).head(n=10)

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,narozeni,umrti
list[str],list[str],list[str],list[str],str,str,str,list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,i64,i64
"[""isni"", ""wikidata""]","[""000000008194381X"", ""Q362154""]","[""1835""]","[""1923""]","""jk01040997""","""Heyduk, Adolf,""","""1835-1923""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,"[""Hejduk, Adolf,""]","[""1835-1923""]",null,"[""1""]",null,null,null,null,null,null,null,null,"[""Narozen 6. 6. 1835 v Rychmburku u Hlinska, zemřel 6. 2. 1923 v Písku. Básník, člen družiny májové, iniciátor česko-slovenských vztahů.""]","[""http://cs.wikipedia.org/wiki/Adolf_Heyduk""]",""" cz a22 n 4500""","""jk01040997""",1835,1923
null,null,"[""1834""]","[""1921""]","""jk01130591""","""Šmídek, Karel,""","""1834-1921""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 7.9.1834 v Bystřici nad Popelkou, zemřel 4.5.1921 v Brně. Pedagog, práce v oboru pedagogiky, historie a filozofie, překlady z němčiny.""]",null,""" nz a22 n 4500""","""jk01130591""",1834,1921
null,null,"[""1828""]","[""1921""]","""jk01131887""","""Švéda, Jan,""","""1828-1921""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,"[""Šwéda, Jan,""]","[""1828-1921""]",null,"[""1""]",null,null,null,null,null,null,null,null,"[""Narozen 9.2.1828 v Českých Budějovicích, zemřel 7.1.1921 v Prachaticích. Msgre., katolický kněz, náboženská literatura.""]",null,""" cz a22 n 4500""","""jk01131887""",1828,1921
"[""isni"", ""wikidata""]","[""0000000116682462"", ""Q12001298""]","[""1832""]","[""1920""]","""jk01072706""","""Lužická, Věnceslava,""","""1832-1920""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,"[""Lužická, Věnceslava Anna,"", ""Srbová, Anna,""]","[""1832-1920"", ""1832-1920""]","[null, ""Skutečné jméno:""]","[""1"", ""1""]",null,null,null,null,"[""Srbová-Lužická, Anna,""]",null,"[""1""]",null,"[""Narozena 6. 12. 1832 v Hořicích, zemřela 4. 5. 1920 v Praze. Kulturní a organizační pracovnice v ženských spolcích, autorka próz s ženskou tematikou, knih pro mládež a výchovných a popularizačních prací, redaktorka, publicistka.""]",null,""" cz a22 n 4500""","""jk01072706""",1832,1920
null,null,"[""1832""]","[""1920""]","""jo2009551991""","""Srbová-Lužická, Anna,""","""1832-1920""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Lužická, Věnceslava,""]","[""Pseudonym:""]","[""1""]",null,"[""Narozena 6. 12. 1832 v Hořicích, zemřela 4. 5. 1920 v Praze. Kulturní a organizační pracovnice v ženských spolcích, autorka próz s ženskou tematikou, knih pro mládež a výchovných a popularizačních prací, redaktorka, publicistka.""]",null,""" nz a22 n 4500""","""jo2009551991""",1832,1920
null,null,"[""1831""]","[""1919""]","""jn20000602550""","""Grohmann, Joseph Virgil,""","""1831-1919""","[""1""]",null,"[""Lipová, Děčín, Česko""]","[""Praha, Česko""]","""Česko""","[""Rakousko-Uhersko""]","[""pedagogika"", ""politika"", ""literatura""]",null,"[""pedagogové"", ""politici"", ""spisovatelé""]","[""muž""]","[""ger""]","[""Grohmann, Josef Virgil,""]","[""1831-1919""]",null,"[""1""]",null,null,null,null,null,null,null,null,"[""Narozen 12. 12. 1831 v Lipové, zemřel 12. 10. 1919 v Praze. Pedagog, politik, německy píšící mytolog. Autor sbírky českých pověstí.""]",null,""" cz a22 n 4500""","""jn20000602550""",1831,1919
null,null,"[""1834""]","[""1917""]","""jk01061617""","""Kopal, Petr,""","""1834-1917""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,

In [38]:
aut.filter(pl.col("narozeni") == 1922).filter(pl.col('umrti').is_null()).sort('narozeni', descending=True)

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,narozeni,umrti
list[str],list[str],list[str],list[str],str,str,str,list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,i64,i64
null,null,"[""1922""]",null,"""jk01022569""","""Dolejš, Svatopluk,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 4.4.1922 v Táboře. JUDr., rozhlasový redaktor, pohádky pro děti, překlad z angličtiny.""]",null,""" cz a22 n 4500""","""jk01022569""",1922,null
null,null,"[""1922""]",null,"""jk01050876""","""Jandera, Alexander,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 22.9.1922. Redaktor, populárně vědecké práce z oboru techniky a lékařství.""]",null,""" cz a22 n 4500""","""jk01050876""",1922,null
null,null,"[""1922""]",null,"""jk01062044""","""Košek, Zdeněk,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen v lednu 1922. PhDr., redaktor, prózy ze severočeského regionu.""]",null,""" nz a22 n 4500""","""jk01062044""",1922,null
null,null,"[""1922""]",null,"""jk01082053""","""Moravec, Ivan,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 1922. Neurolog. Překládá ze švédštiny. Napsal memoáry.""]",null,""" cz a22 n 4500""","""jk01082053""",1922,null
null,null,"[""1922""]",null,"""jk01101293""","""Prokešová, Vlasta,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozena 8.3.1922 v Opočně. Lékárenská laborantka, prozaička, básnířka.""]",null,""" nz a22 n 4500""","""jk01101293""",1922,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,"[""1922""]",null,"""mzk2004252223""","""Plšková, Emilie,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozena 20.8.1922 v Praze. Spisovatelka.""]",null,""" nz a22 n 4500""","""mzk2004252223""",1922,null
null,null,"[""1922""]",null,"""ola2006357541""","""Němec, Ladislav,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 1922 v Chrlicích. Mgr., hudební pedagog a skladatel, varhaník, soustřeďuje se na tvorbu pro děti, též autor říkadel pro děti.""]",null,""" cz a22 n 4500""","""ola2006357541""",1922,null
null,null,"[""1922""]",null,"""mzk2008473045""","""Horek, Vladimír,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 1922. Autobiografický příběh z období totálního nasazení.""]",null,""" cz a22 n 4500""","""mzk2008473045""",1922,null


In [39]:
aut.filter(pl.col("narozeni") < 1923).filter(pl.col('umrti').is_null()).sort('narozeni', descending=True).head(n=10)

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,narozeni,umrti
list[str],list[str],list[str],list[str],str,str,str,list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,i64,i64
null,null,"[""1922""]",null,"""jk01022569""","""Dolejš, Svatopluk,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 4.4.1922 v Táboře. JUDr., rozhlasový redaktor, pohádky pro děti, překlad z angličtiny.""]",null,""" cz a22 n 4500""","""jk01022569""",1922,null
null,null,"[""1922""]",null,"""jk01050876""","""Jandera, Alexander,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 22.9.1922. Redaktor, populárně vědecké práce z oboru techniky a lékařství.""]",null,""" cz a22 n 4500""","""jk01050876""",1922,null
null,null,"[""1922""]",null,"""jk01062044""","""Košek, Zdeněk,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen v lednu 1922. PhDr., redaktor, prózy ze severočeského regionu.""]",null,""" nz a22 n 4500""","""jk01062044""",1922,null
null,null,"[""1922""]",null,"""jk01082053""","""Moravec, Ivan,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 1922. Neurolog. Překládá ze švédštiny. Napsal memoáry.""]",null,""" cz a22 n 4500""","""jk01082053""",1922,null
null,null,"[""1922""]",null,"""jk01101293""","""Prokešová, Vlasta,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozena 8.3.1922 v Opočně. Lékárenská laborantka, prozaička, básnířka.""]",null,""" nz a22 n 4500""","""jk01101293""",1922,null
null,null,"[""1922""]",null,"""jk01101870""","""Rada, Josef,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 13. 7. 1922 v Chlebech na Benešovsku. Prof. Ing., CSc., vysokoškolský pedagog zaměřený na elektrotechniku, též autor poezie.""]",null,""" cz a22 n 4500""","""jk01101870""",1922,null
null,null,"[""1922""]",null,"""jk01120198""","""Smutný, Svatopluk,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen roku 1922 v Praze. Novinář, redaktor Lidové demokracie a časopisů Nazdar a Cyklistika. Práce v oboru sportu.""]",null,""" nz a22 n 4500""","""jk01120198""",1922,null
null,null,"[""1922""]",null,"""jk01131989""","""Táborský, Jiří,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Novinář, překlady z ruštiny, cestopis.""]",null,""" cz a22 n 4500""","""jk01131989""",1922,null
null,null,"[""1922""]",null,"""jk01151327""","""Wagner, Miloslav,""","""1922-""","[""1""]",null,null,null,"""Česko""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 18.2.1922 v Hradci Králové. PhDr., rusista, redaktor časopisu Československá rusistika. Práce v oboru, publikovány též časopisecky.""]",null,""" nz a22 n 4500""","""jk01151327""",1922,null


In [23]:
stoleti = aut.filter(pl.col("narozeni") == 1922).filter(pl.col('umrti').is_null()).select(pl.col('100_7')).drop_nulls().to_series().to_list()
df.filter(pl.col("100_7").is_in(stoleti)).group_by("autorstvo").len().sort(by="len", descending=True).head(10)

autorstvo,len
str,u32
"""Horek Vladimír""",8
"""Holota Benedikt""",2
"""Plšková Emilie""",1
"""Šťastný Zdeněk""",1
"""Palka Jiří""",1
"""Šabatová Anna""",1
"""Smutný Svatopluk""",1
"""Preisler Karel""",1
"""Macháčková Vlasta""",1


In [25]:
cervi_dira = pl.DataFrame([
    {'jmeno': 'Karel Hynek Mácha', 'narozeni': datetime.date(year=1810, month=11, day=16), 'umrti': datetime.date(year=1836, month=11, day=6)},
    {'jmeno': 'Adolf Heyduk', 'narozeni': datetime.date(year=1835, day=6, month=6), 'umrti': datetime.date(year=1923, day=6, month=2)},
    {'jmeno': 'Vladimír Benedikt Holota', 'narozeni': datetime.date(year=1922, month=8, day=5), 'umrti': datetime.date(year=2025, day=31, month=12)}
]).with_columns()

In [43]:
cervi_dira

jmeno,narozeni,umrti
str,date,date
"""Karel Hynek Mácha""",1810-11-16,1836-11-06
"""Adolf Heyduk""",1835-06-06,1923-02-06
"""Vladimír Benedikt Holota""",1922-08-05,2025-12-31


In [51]:
bar1 = alt.Chart(cervi_dira.head(2).to_pandas(), title=alt.Title("Od Máje nás dělí jen dva životy…",subtitle='Čáry pokrývají dobu mezi narozením a úmrtím autora.'), width=250).mark_bar(height=10).encode(
    x=alt.X('narozeni:T', axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1800},{'year':2020}])).title(None),
    x2='umrti:T',
    y=alt.Y('jmeno:N', sort=cervi_dira.select(pl.col('jmeno')).to_series().to_list()).title(None),
color=alt.Color('jmeno:N', legend=None, scale=alt.Scale(range=['#E09DA3'])))
bar2 = alt.Chart(cervi_dira.tail(1).to_pandas(), title="Dvě století české literatury ve dvou životech").mark_bar(height=10, cornerRadiusTopRight=10, cornerRadiusBottomRight=10).encode(
    x=alt.X('narozeni:T').title(None),
    x2='umrti:T',
    y=alt.Y('jmeno:N', axis=alt.Axis(orient='left', domainOpacity=0, tickColor='white'), sort=cervi_dira.select(pl.col('jmeno')).to_series().to_list()).title(None),
color=alt.Color('jmeno:N', legend=None, scale=alt.Scale(range=['#E09DA3'])))
zivoty = (bar1 + bar2)
zivoty

alt.LayerChart(...)

In [31]:
def hledej_autorstvo(label, keywords):
        print(keywords)
        return df.filter(pl.col('rok') > 1800).filter(pl.col('100_7') == keywords).with_columns(kdo = pl.lit(label)).with_columns(
                   pl.col("rok").map_elements(
                       lambda x: datetime.date(year=int(x), month=1, day=1), 
                       return_dtype=pl.Date
                   ).cast(pl.Datetime)
               )

In [33]:
koho_ukazujeme = {'Karel Hynek Mácha':'jk01072915','Petr Miloslav Veselský': 'jk01142116', 'Josef Kalenský': 'jk01052646', 'Fan Vavřincová': 'jk01141829'}

In [35]:
y_encoding = {
        'field': 'kdo',
        'type': 'nominal',
        'title': None,
        'sort': [jmeno for jmeno,kod in koho_ukazujeme.items()]
    }

In [37]:
from datetime import date

In [39]:
do_grafu = pl.concat([hledej_autorstvo(jmeno, kod) for jmeno, kod in koho_ukazujeme.items()])
do_grafu = do_grafu.filter(
    (pl.col("kdo") == "Karel Hynek Mácha") & (pl.col('rok') <= date(1836, 12, 31)) |
    (pl.col("kdo") == "Petr Miloslav Veselský") & (pl.col('rok') <= date(1889, 12, 31)) |
    (pl.col("kdo") == "Josef Kalenský") & (pl.col('rok') <= date(1951, 12, 31)) |
    (pl.col("kdo") == "Fan Vavřincová") & (pl.col('rok') <= date(2012, 12, 31))
)
do_grafu

jk01072915
jk01142116
jk01052646
jk01141829


rok,titul,autorstvo,100_7,001,kdo
datetime[μs],str,str,str,str,str
1836-01-01 00:00:00,"""Mág""","""Mácha Karel Hynek""","""jk01072915""","""cpk20041417516""","""Karel Hynek Mácha"""
1834-01-01 00:00:00,"""Jozefka, anebo, Podiwné schled…","""Veselský Petr Miloslav""","""jk01142116""","""cpk20021123969""","""Petr Miloslav Veselský"""
1834-01-01 00:00:00,"""Ukradený střewjc, anebo, Prawý…","""Veselský Petr Miloslav""","""jk01142116""","""nkc20102130003""","""Petr Miloslav Veselský"""
1837-01-01 00:00:00,"""Jolanta, anebo, Obnowená důwěr…","""Veselský Petr Miloslav""","""jk01142116""","""nkc20092009453""","""Petr Miloslav Veselský"""
1838-01-01 00:00:00,"""Mocnost lásky, neb, Wiljm a Ma…","""Veselský Petr Miloslav""","""jk01142116""","""cpk20021195155""","""Petr Miloslav Veselský"""
1855-01-01 00:00:00,"""Jolanta, aneb, Obnowená důwěra""","""Veselský Petr Miloslav""","""jk01142116""","""bknaus19240""","""Petr Miloslav Veselský"""
1866-01-01 00:00:00,"""Čarodějnice, aneb, Bůh dopustí…","""Veselský Petr Miloslav""","""jk01142116""","""nkc20102032126""","""Petr Miloslav Veselský"""
1889-01-01 00:00:00,"""Karla Havlíčka kutnohorský pob…","""Veselský Petr Miloslav""","""jk01142116""","""bknaus19241""","""Petr Miloslav Veselský"""
1889-01-01 00:00:00,"""Národní pohádky a báje""","""Kalenský Josef""","""jk01052646""","""nkc20142542424""","""Josef Kalenský"""


In [49]:
tri_kariery = alt.Chart(do_grafu.to_pandas(), title=alt.TitleParams("…nebo tři a čtvrt kariéry", anchor="start", subtitle="Tečky označují knihy vydané během života.", dx=-12), width=250).mark_circle(size=15) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1800},{'year':2020}])), 
            y=alt.Y(**y_encoding, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='white')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[9, 11])), 
            color=alt.Color('kdo:N', scale=alt.Scale(range=['#D6534B']), 
                            sort=[jmeno for jmeno, kod in koho_ukazujeme.items()]).legend(None)) \
        .transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

tri_kariery

alt.Chart(...)

In [55]:
zivoty_a_kariery = alt.vconcat(zivoty, tri_kariery).resolve_scale(color='independent',y="independent").resolve_axis(y='shared').configure_view(stroke='transparent')
zivoty_a_kariery

alt.VConcatChart(...)

In [57]:
me_to_neurazi(zivoty_a_kariery, soubor="02_cervi_diry", kredity=kredity['default'])

<figure><a href="https://data.irozhlas.cz/knihy-grafy/02_cervi_diry.svg" target="_blank"><img src="https://data.irozhlas.cz/knihy-grafy/02_cervi_diry.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." /></a></figure>
<figure><a href="https://michalkasparek.cz/sklad/02_cervi_diry.svg" target="_blank"><img src="https://michalkasparek.cz/sklad/02_cervi_diry.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." /></a></figure>
